# LangChain ReAct agent

In [ ]:
! pip3 install openai
! pip3 install langchain

In [ ]:
AZURE_OPENAI_API_VERSION = ""
AZURE_OPENAI_API_KEY = ""
AZURE_OPENAI_API_BASE = ""
AZURE_DEPLOYMENT_NAME = ""

In [2]:
from agentquest.drivers.mastermind import MasterMindDriver
from agentquest.utils import Action, load_data
from termcolor import cprint

Load a generic 4 digits mastermind game

In [3]:
game = load_data('mastermind', '4 digits')[0]

Initialize the prompt and the ReAct agent

In [ ]:
from langchain.agents import create_react_agent
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import StringPromptTemplate
from langchain.tools.base import BaseTool
from typing import List

prompt = """
You are tasked to play the Mastermind game. The host chooses a number and gives 
you the amount of digits. You have to gess the correct number as fast as you can.

Rules:
1. At each round you provide a number as guess. The hosts provides you the number 
of correct digits in the wrong position and the number of correct digits in the 
correct position.
2. The game ends when the hosts outputs 'You Won!'
3. The number can contain repetitions.
4. The number can contain any possible digit.
5. Avoid bruteforce.

Use the following tools:
{tools}

Use the following format:

Thought: summarize the info in the last observation and think about what to do.
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Final Answer: always provide the final answer or summarise what you did. Remember the keyword 'Final Answer' before

Input task: {input}

{agent_scratchpad}
"""

llm = AzureChatOpenAI(
    openai_api_type='azure',
    openai_api_version=AZURE_OPENAI_API_VERSION,
    openai_api_key=AZURE_OPENAI_API_KEY,
    openai_api_base=AZURE_OPENAI_API_BASE,
    deployment_name=AZURE_DEPLOYMENT_NAME,
    temperature=0
)

class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[BaseTool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "

        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        #print(thoughts)
        
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in self.tools]
        )
        
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])

        return self.template.format(**kwargs)

# Define a dummy tool for closed-box environments
class ProxyTool(BaseTool):
    name = "proxytool"
    description = "Provide the action you want to perform"
    def _run(self):
        pass

In [5]:
prompt = CustomPromptTemplate(
    template=prompt,
    tools=[ProxyTool()],
    input_variables=[
        "input", 
        "intermediate_steps", 
        "tools", 
        "tool_names", 
        "agent_scratchpad"
    ],
)
agent = create_react_agent(llm, [ProxyTool()], prompt)
intermediate_steps = []

Initialize the mastermind driver and get the first observation

In [6]:
driver = MasterMindDriver(game)
obs = driver.reset() # Get the first observation
print(f'OBSERVATION: {obs.output}')

OBSERVATION: Start guessing the 4 digits number.


Start the Agent loop solving the task

In [7]:
step_cnt = 0
while not obs.done and step_cnt<10:
    # Retrieve the agent output
    agent_choice = agent.invoke(
        {'input': obs.output,
        'intermediate_steps': intermediate_steps}
    )

    action = Action(action_value=agent_choice.tool_input)
    print(action.action_value)
    # Perform the step
    obs = driver.step(action)
    print(obs.output)
    # Update intermediate steps
    intermediate_steps.append((agent_choice, obs.output))

    step_cnt+=1

1234

Your guess has 1 correct numbers in the wrong position and 0 correct numbers in the correct position. Keep guessing.
1256

Your guess has 2 correct numbers in the wrong position and 0 correct numbers in the correct position. Keep guessing.
1789

Your guess has 3 correct numbers in the wrong position and 0 correct numbers in the correct position. Keep guessing.
7812

Your guess has 1 correct numbers in the wrong position and 1 correct numbers in the correct position. Keep guessing.
2178

Your guess has 1 correct numbers in the wrong position and 1 correct numbers in the correct position. Keep guessing.
2718

Your guess has 0 correct numbers in the wrong position and 2 correct numbers in the correct position. Keep guessing.
2713

Your guess has 0 correct numbers in the wrong position and 1 correct numbers in the correct position. Keep guessing.
2173

Your guess has 1 correct numbers in the wrong position and 0 correct numbers in the correct position. Keep guessing.
2711

Your guess